## Assignment Eleven

## Due May 8th

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spqd
from time import time
%matplotlib inline

In [2]:
def simpson_method(a,b,N,f):
    Nint = int(N)
    Tint = 2*Nint
    xvals = np.linspace(a,b,Tint+1)
    fvals = f(xvals)
    dx = (b-a)/(2.*N)
    return dx/3.*(fvals[0] + fvals[Tint] + 2.*np.sum(fvals[2:Tint-1:2]) + 4.*np.sum(fvals[1:Tint:2])) 

In [3]:
def adap_quad_comp(f,a,b,c,fa,fb,fc,Aab,tol):    
    d = (a+c)/2.
    e = (b+c)/2.
    fd = f(d)
    fe = f(e)
    dx3 = (d-a)/3.
    Aac = dx3*(fa+4.*fd+fc)
    Acb = dx3*(fc+4.*fe+fb)
    if np.abs(Aab-(Aac+Acb))<=tol: 
        return (16./15.*(Aac+Acb)-Aab/15.)
    else:  
        F1 = adap_quad_comp(f,a,c,d,fa,fc,fd,Aac,tol)
        F2 = adap_quad_comp(f,c,b,e,fc,fb,fe,Acb,tol)
        return F1 + F2
    
def adap_quad(f,a,b,tol):
    c = (a+b)/2.
    fa = f(a)
    fb = f(b)
    fc = f(c)
    dx3 = (c-a)/3.
    Aab = dx3*(fa+4.*fc+fb)
    ival = adap_quad_comp(f,a,b,c,fa,fb,fc,Aab,tol)    
    return ival

**Problem 1**: [7 points] Use the adaptive quadrature algorithm to evaluate 

$$
\int_{-\pi/4}^{\pi/2} \frac{dt}{\sin\left(\sqrt{|t|} \right)}.
$$

State how you arrive at your result i.e. what tolerances do you choose and what degree of accuracy can you argue that they provide you?  Why do you not encounter divide-by-zero difficulties at $t=0$?

In [4]:
#1. 
f = lambda t: 1. / (np.sin(np.sqrt(np.abs(t))))
a = -np.pi / 4.
b = np.pi / 2. 
tol1 = 1e-4
tol2 = 1e-6
tol3 = 1e-8
print adap_quad(f, a, b, tol1)
print adap_quad(f, a, b, tol2)
print adap_quad(f, a, b, tol3)

4.6068607202104825
4.606990987173205
4.606992366667309


**Problem 2**: [10 points] Definite integrals sometimes have the property that the integrand becomes infinite at one or both of the endpoints, but the integral itself is finite.  In other words, $\lim_{x\rightarrow a}|f(x)|=\infty$ or $\lim_{x\rightarrow a}|f(x)|=\infty$, but 

$$
\int_{a}^{b}f(x)dx 
$$

exists and is finite.  

* [7 points] Modify the adaptive quadrature algorithm so that if an infinite value of $f(a)$ or $f(b)$ is detected, an appropriate warning message is displayed and $f(x)$ is reevaluated at a point very near to a or b.  

* [3 points] Find an example that triggers the warning, but has a finite integral.  

In [30]:
def adap_quad(f,a,b,tol):
    c = (a+b)/2.
    
    try: 
        fa = f(a)
    except: 
        print "Error: f(a) approaches infnity."
        s = input("Enter a new 'a' value: ")
        print type(s)
        a = float(s)
        fa = f(a)
        
    try:
        fb = f(b)
    except:
        s = input("Enter a new 'a' value: ")
        print type(s)
        b = float(s)
        fb = f(b)
        
    fc = f(c)
    dx3 = (c-a)/3.
    Aab = dx3*(fa+4.*fc+fb)
    ival = adap_quad_comp(f,a,b,c,fa,fb,fc,Aab,tol)    
    return ival

In [32]:
f1 = lambda x: 1. / (2 - x)
a1 = 0
b1 = 2  
print adap_quad(f1, a1, b1, 1e-4)

Enter a new 'a' value: 1.8
<type 'float'>
2.30258726596


**Problem 3**: (23 pts) Consider the following table:

| $x$  | 1 | 1.1 | 1.2| 1.3  | 1.4 |
| ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| $\cos(x)$ | 0.54030  | 0.45360  |0.36236 | 0.26750 | 0.16997  |

(i) [10 points] Using Newton Interpolation, write a program to obtain

(ii) [2 points] A polynomial of degree 1 that interpolates $\cos(x)$.

(iii) [2 points] A polynomial of degree 2 that interpolates $\cos(x)$.

(iv) [2 points] A polynomial of degree 3 that interpolates $\cos(x)$.

(v) [2 points] A polynomial of degree 4 that interpolates $\cos(x)$.

(vi) [5 points] Compute $\cos(1.05)$ using the various order polynomials and investigate the error for the Newton polynomial approximations in (ii) - (v)

In [50]:
def newton_interpolation(degree):
    xIinterval = np.linspace(1., 1.+(degree/10.), int(degree+1))
    fx = np.cos(x)
    p = 1.
    a1 = []
    a2 = []
    a3 = []
    a4 = []
    if degree >= 1:
        while p <= degree:
            a1.append((f[p] - f[p-1]) / (xIinterval[p] - xIinterval[p-1]))
            p += 1
    if degree >= 2:
        p = 2
        while p <= degree:
            a2.append((a1[p-1] - a1[p-2]) / (xIinterval[p] - xIinterval[p-2]))
            p+=1
    if degree >= 3:
        p = 3
        while p <= degree:
            a3.append((a2[p-2] - a2[p-3]) / (xIinterval[p] - xIinterval[p-3])
            p += 1
    if degree >= 4:
        p = 4
        while p <= degree:
            a4.append((a3[p-3] - a3[p-4]) / xIinterval[p] - xIinterval[p-4])
            p += 1
    if degree == 1:
        f = lambda x: fx[0] + a1[0]*(x-xIinterval[0])
        return f
    if degree == 2:
        f = lambda x: fx[0] + a1[0]*(x-xInterval[0]) + a2[0]*(x-xInterval[0])*(x-xInterval[1])
        return f
    if degree == 3:
        f = lambda x: fx[0] + a1[0]*(x-xInterval[0]) + a2[0]*(x-xInterval[0])*(x-xInterval[1]) + a3[0]*(x-xInterval[0])*(x-xInterval[1])
        return f  
    if degree == 4:
        f = lambda x: fx[0] + a1[0]*(x-xInterval[0]) + a2[0]*(x-xInterval[0])*(x-xInterval[1]) + a3[0]*(x-xInterval[0])*(x-xInterval[1]) = a4[0]*(x-xInvetcal[0])*(x-xInterval[1])
        return f
    else: 
        return "Enter degrees 1-4 only!"

SyntaxError: invalid syntax (<ipython-input-50-2bfb8773d947>, line 22)

In [49]:
d1 = newton_interpolation(1)
d2 = newton_interpolation(2)
d3 = newton_interpolation(3)
d4 = newton_interpolation(4)

ac = np.cos(1.05)
ad1 = d1(1.05)
ad2 = d2(1.05)
ad3 = d3(1.05)
ad4 = d4(1.05)

e1 = np.abs(ad1 - ac) / ac
e2 = np.abs(ad2 - ac) / ac
e3 = np.abs(ad3 - ac) / ac
e4 = np.abs(ad4 - ac) / ac

print ac, ad1, ad2, ad3, ad4
print e1, e2, e3, e4

NameError: name 'newton_interpolation' is not defined

**Problem 4**: (10 pts) 
    
i) [8 points] Write a python program to find the **cubic root** of a number based on Newton method.

ii) [2 points] Using your program in i), compute the root of $a=155$

In [45]:
f = lambda x, a: x*x*x-a
fp = lambda x: 3.0*x**2

In [46]:
def cubic_root_newton_method(x, a):
    h = f(x, a) / fp(x)
    while np.abs(h) >= .0001:
        h = f(x, a) / fp(x)
        x = x-h
    print "The value of the root is: " + str(x)

In [37]:
a = 155
cubic_root_newton_method(a/2, a)

The value of the root is: 5.37168535494
